In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # <-- Correct import here!
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from tqdm import tqdm
import numpy as np
import os

# Load Data
train_df = pd.read_csv('../data/train.csv')  # Replace with your path
dev_df = pd.read_csv('../data/dev.csv')

# PyTorch-ready dataset
class RedditDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts              # Store the list of comment texts
        self.labels = labels            # Store the corresponding list of integer labels
        self.tokenizer = tokenizer      # Store the tokenizer (e.g., RobertaTokenizer)
        self.max_length = max_length    # Store the maximum sequence length

    def __len__(self):
        return len(self.texts)  # Important: this allows DataLoader to know dataset size

    def __getitem__(self, idx):
        """
        Fetches a single sample from the dataset.

        Args:
            idx (int): Index of the item.

        Returns:
            dict: A dictionary containing:
                - 'input_ids': Raw text must be converted into numbers because models can't understand plain words.
                - 'attention_mask': A list that tells the model which tokens are real (1) and which ones are just padding (0).
                - 'labels': The label 

        * Tensor is a multi-dimensional array (like a super-powered NumPy array) that can run on a GPU very efficiently

        ex: 
        Raw Text:
        "eating soap to own the republicans"

        Tokenized to IDs:
        [0, 1553, 4153, 7, 1225, 5, 13815, 2, *1, *1]

        Attention Mask:
        [1, 1, 1, 1, 1, 1, 1, 1, *0, *0]

        Label:
        0

        Models usually expect inputs to have the same length (max = max_lenght).
        If your token list is too short, you add padding tokens (which are just zeros).
        """
        # Fetch the text and its corresponding label using the index
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize the text:
        # - truncation: cuts off texts longer than max_length
        # - padding: adds padding tokens to shorter texts
        # - return_tensors='pt': returns PyTorch tensors instead of lists
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length = self.max_length,
            return_tensors='pt' # stands for PyTorch
        )

        # Return a dictionary with model inputs and label
        return {
            'input_ids': encoding['input_ids'].squeeze(),         # Tensor of input token IDs .squeeze() again just removes the extra dimension.
            'attention_mask': encoding['attention_mask'].squeeze(), # Tensor indicating real tokens vs padding
            'labels': torch.tensor(label, dtype=torch.long)        # Make sure it becomes a PyTorch Tensor, because the model expects labels to be Tensors during training.
        }

        """
        squeeze example:

        [
        [ 0, 1553, 4153, 7, 1225, 5, 13815, 2, 1, 1 ]
        ]
        (batch size 1, 10 tokens)

        to

        [ 0, 1553, 4153, 7, 1225, 5, 13815, 2, 1, 1 ]
        (just 10 tokens)

        """

In [ ]:
# -------------------------
# Data preprocessing
# -------------------------
# Encode labels to integers
labels = train_df['label'].unique()
label2id = {label: idx for idx, label in enumerate(labels)} # {"label": 'id'}
id2label = {idx: label for label, idx in label2id.items()} # {'id':'label}


train_df['label_id'] = train_df['label'].map(label2id)
dev_df['label_id'] = dev_df['label'].map(label2id)

# -------------------
# Prepare DataLoaders
# -------------------
# CONFIG
MODEL_NAME = 'roberta-base'
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = 4
LEARNING_RATE = 2e-5
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

# Prepare dataset for PyTorch (processes the raw data)
train_dataset = RedditDataset(train_df['text'].tolist(), train_df['label_id'].tolist(), tokenizer, MAX_LENGTH)
dev_dataset = RedditDataset(dev_df['text'].tolist(), dev_df['label_id'].tolist(), tokenizer, MAX_LENGTH)

# Create small batches (batches them without changing the data itself)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE)

# Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cuda is faster but it requires GPU
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))
model.to(device) # Moves all model weights and computations to the device you selected (cuda if GPU, otherwise cpu)

# Optimizer and Scheduler
# model.parameters() are all the learnable weights in the model.
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE) # A type of optimizer that updates the model's weights to minimize the loss. 
total_steps = len(train_loader) * EPOCHS 
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps) # It controls how the learning rate changes during training. In our case it linearly decreases 


# Training Loop
def train(model, loader):
    model.train() # Sets the model into training mode. It should be done before training starts
    total_loss = 0
    for batch in tqdm(loader):
        optimizer.zero_grad() # Reset gradients: every time before a new update, clear the old gradients.

        # Move data to device (Model and data must be on the same device to avoid errors)
        input_ids = batch['input_ids'].to(device) 
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass (Feed the inputs into the model)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()

        # Update model weights based on the gradients
        optimizer.step()
        scheduler.step()

        # Track total loss (for reporting)
        total_loss += loss.item()

    return total_loss / len(loader) # average loss per batch

# Evaluation Function
def evaluate(model, loader):
    model.eval() # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    with torch.no_grad(): # Turns off gradient calculations (Saves memory and speeds up evaluation because we don't need gradients)
        for batch in tqdm(loader):
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            # Forward pass 
            outputs = model(input_ids=input_ids, attention_mask=attention_mask) 
            # Get predictions (pick the class with the highest score (probability) for each example in the batch)
            preds = torch.argmax(outputs.logits, dim=1) # outputs.logits = raw scores for each class
            # Save predictions and true labels
            # We Move predictions and labels from GPU to CPU and convert them into numpy arrays
            # because some Python operations (like NumPy or scikit-learn) cannot work directly with GPU Tensors.
            all_preds.extend(preds.cpu().numpy()) 
            all_labels.extend(labels.cpu().numpy())

    # Take the F1 score for each label (class) separately, and then average them equally (It does not care if one label has 1000 examples and another class has 100 examples)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    return macro_f1

# Run Training
best_f1 = 0
best_model_state = None

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    train_loss = train(model, train_loader)
    val_f1 = evaluate(model, dev_loader)
    print(f"Train Loss: {train_loss:.4f}, Validation Macro F1: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        best_model_state = model.state_dict()

# # Loads back the best version of the model that you saved during training (Forget your current weights — load these saved weights instead)
# model.load_state_dict(best_model_state) 

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4


  0%|          | 0/2000 [00:00<?, ?it/s]

forward pass done
backward pass done


  0%|          | 1/2000 [00:26<14:44:33, 26.55s/it]

1.595721960067749


  0%|          | 1/2000 [00:31<17:23:37, 31.32s/it]


KeyboardInterrupt: 

In [ ]:
# Predict on Test Set
test_df = pd.read_csv('../data/test.csv')
test_texts = test_df['text'].tolist()
test_ids = test_df['id'].tolist()

# PyTorch Dataset
test_dataset = RedditDataset(test_texts, [0]*len(test_texts), tokenizer, MAX_LENGTH)
# Batch
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
# Set the model to evaluation mode
model.eval()
all_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        # Move data to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # Forward pass and get predictions
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())

pred_labels = [id2label[p] for p in all_preds]

# Save Submission File
output_df = pd.DataFrame({'id': test_ids, 'label': pred_labels})
os.makedirs('outputs', exist_ok=True)
output_df.to_csv('outputs/track_2_test.csv', index=False)

print("Saved submission to outputs/track_2_test.csv")